## Get bugs information from `.csv` files

The following part extracts information from the `.csv` files dumped during running test suites on DBMSs.

In [1]:
import pandas as pd

bugs_df = pd.read_csv("output/demo_bugs.csv")
logs_df = pd.read_csv("output/demo_logs.csv")

print(bugs_df.info())
print(logs_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       119 non-null    int64 
 1   TESTFILE_INDEX   119 non-null    int64 
 2   TESTFILE_PATH    119 non-null    object
 3   TESTCASE_INDEX   119 non-null    int64 
 4   ERROR_SQL        119 non-null    object
 5   CASE_TYPE        119 non-null    object
 6   EXPECTED_RESULT  119 non-null    object
 7   ACTUAL_RESULT    117 non-null    object
 8   DATE             119 non-null    object
dtypes: int64(3), object(6)
memory usage: 8.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  119 non-null    int64 
 1   BUGS_INDEX  119 non-null    int64 
 2   LOGS        119 non-null    object
dtypes: int64(2), object(1)
memory usage: 

In [2]:
from random import randint

id = randint(0, len(bugs_df))
bug_row = bugs_df.loc[id]
log_row = logs_df.loc[id]
dbms_name = 'duckdb'  # TODO need add a dbms_name field in the dataframe
test_suite = 'sqlite_tests'  # TODO same as above


## Reproduce an error testcase

The following code picks a bug record in the `.csv` file and run on the selected db. Maybe running using the testrunner.

In [3]:
from src import testrunner

if dbms_name == 'sqlite':
    r = testrunner.SQLiteRunner()
elif dbms_name == 'duckdb':
    r = testrunner.DuckDBRunner()
elif dbms_name == 'cockroachdb':
    r = testrunner.CockroachDBRunner()
elif dbms_name == 'mysql':
    r = testrunner.MySQLRunner()
else:
    exit("Not implement yet")
    
r.connect(":memory:")

print(bug_row)
print(log_row)
r.executemany_stmt(log_row.LOGS)


print("The Expected Result is:")
print(bug_row.EXPECTED_RESULT)
print("The Actual Result is:")
result = r.execute_query(bug_row.ERROR_SQL)
for row in result:
    for item in row:
        print(item)






Unnamed: 0                                                        37
TESTFILE_INDEX                                                   320
TESTFILE_PATH              sqlite_tests/random/expr/slt_good_29.test
TESTCASE_INDEX                                                  2060
ERROR_SQL          SELECT DISTINCT - CASE CAST ( - 91 AS INTEGER ...
CASE_TYPE                                                      Query
EXPECTED_RESULT                                                   65
ACTUAL_RESULT                                                   2104
DATE                                                  22-10-26-12:53
Name: 37, dtype: object
Unnamed: 0                                                   37
BUGS_INDEX                                                   37
LOGS          CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
Name: 37, dtype: object
The Expected Result is:
65
The Actual Result is:
2104.5081967213114
